In [1]:
!ls

alpaca-bitcoin-sentiment-dataset.json  llama_models	    prueba.py
bitcoin-sentiment-tweets.csv	       llama_promt_log.txt  README.md
data				       model		    requirements.txt
Error_intall.log		       not_git		    samples
fintuning.ipynb			       promts.log	    tranfor_data.py
get_file.py			       prueba_2.py	    version.md
llama				       prueba_3.py
llama-cpp-python		       prueba_4.py


In [2]:
import json
def get_file(path_file):
    with open(path_file) as f:
        data = json.load(f)
    print(data.keys())
    return ('\n\n').join([f"{str(key)}: {data[key]}" for key in data.keys() if key != 'ground_truth'])
    
print(get_file('./data/dummy.json'))

dict_keys(['Título', 'Cuerpo', 'ground_truth'])
Título: VisComp Papers

Cuerpo: Date: Thu, 21 Nov 1996 20:09:30 GMT Server: NCSA/1.5 Content-type: text/html Last-modified: Thu, 24 Aug 1995 08:39:01 GMT Content-length: 4395 VisComp Papers Ramesh Jain Director Visual Computing Laboratory University of California, San Diego 9500 Gilman Drive, Mail Code 0407 La Jolla, CA 92093-0407 Ramesh Jain is currently a Professor of Electrical and Computer Engineering , and Computer Science and Engineering at University of California, San Diego . Before joining UCSD, he was a Professor of Electrical Engineering and Computer Science , and the founding Director of the Artificial Intelligence Laboratory at the University of Michigan , Ann Arbor , MI . He has also been affiliated with Stanford University , IBM Almaden Research Labs , General Motors Research Labs, Wayne State University , University of Texas at Austin , University of Hamburg , Germany , and Indian Institute of Technology, Kharagpur , India

Intalamos las dependencias necesarias

- transformers: for loading a large language model and fine-tuning it.

- bitsandbytes: for loading the model in 4-bit precision.

- accelerate: for training models and performing inference at scale.

- peft: for fine-tuning a small number of parameters.

- trl: for training transformer language models using Reinforcement Learning.


In [3]:
!pip install -q accelerate==0.21.0 --progress-bar off
!pip install -q peft==0.4.0 --progress-bar off
!pip install -q bitsandbytes==0.40.2 --progress-bar off
!pip install -q transformers==4.31.0 --progress-bar off
!pip install -q trl==0.4.7 --progress-bar off


In [4]:
!pip install -q torch torchvision torchaudio --progress-bar off

In [5]:
!pip install -q scipy --progress-bar off

In [6]:
!pip list | grep tor

decorator                 5.1.1
rfc3339-validator         0.1.4
rfc3986-validator         0.1.1
torch                     2.1.0
torchaudio                2.1.0
torchvision               0.16.0
tornado                   6.3.3


In [7]:
import sys
sys.path

['/home/alourido/Desktop/alpaca-classification',
 '/home/alourido/anaconda3/lib/python311.zip',
 '/home/alourido/anaconda3/lib/python3.11',
 '/home/alourido/anaconda3/lib/python3.11/lib-dynload',
 '',
 '/home/alourido/anaconda3/lib/python3.11/site-packages']

In [8]:
# Nueva lista de directorios
new_sys_path = [
    '',
    '/home/alourido/anaconda3/envs/alpaca/lib/python311.zip',
    '/home/alourido/anaconda3/envs/alpaca/lib/python3.11',
    '/home/alourido/anaconda3/envs/alpaca/lib/python3.11/lib-dynload',
    '/home/alourido/anaconda3/envs/alpaca/lib/python3.11/site-packages',
    '/home/alourido/Desktop/llama'
]

# Reemplaza sys.path con la nueva lista de directorios
sys.path = new_sys_path

In [9]:
import torch
torch.cuda.is_available() 

True

A continuación, cargaremos las bibliotecas necesarias para ajustar un modelo de lenguaje grande (LLM) como Llama 2. Examinaremos cada clase importada con mayor detalle en las secciones siguientes.

In [10]:
import os
from random import randrange
from functools import partial
import torch
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForLanguageModeling,
                          EarlyStoppingCallback,
                          pipeline,
                          logging,
                          set_seed)

import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM
from trl import SFTTrainer

In [11]:
import pandas as pd

In [12]:
!pip install -q gdown --progress-bar off
!gdown 1xQ89cpZCnafsW5T3G3ZQWvR7q682t2BN

Downloading...
From: https://drive.google.com/uc?id=1xQ89cpZCnafsW5T3G3ZQWvR7q682t2BN
To: /home/alourido/Desktop/alpaca-classification/bitcoin-sentiment-tweets.csv
100%|█████████████████████████████████████████| 242k/242k [00:01<00:00, 234kB/s]


In [13]:
path_data='./data/bitcoin-sentiment-tweets.csv'
df = pd.read_csv(path_data)
df.head()

,date,tweet,sentiment
0,Fri Mar 23 00:40:40 +0000 2018,@p0nd3ea Bitcoin wasn't built to live on excha...,1.0
1,Fri Mar 23 00:40:40 +0000 2018,@historyinflicks Buddy if I had whatever serie...,1.0
2,Fri Mar 23 00:40:42 +0000 2018,@eatBCH @Bitcoin @signalapp @myWickr @Samsung ...,0.0
3,Fri Mar 23 00:41:04 +0000 2018,@aantonop Even if Bitcoin crash tomorrow morni...,0.0
4,Fri Mar 23 00:41:07 +0000 2018,I am experimenting whether I can live only wit...,1.0


In [14]:
def sentiment_score_to_name(score: float):
    if score > 0:
        return "Positive"
    elif score < 0:
        return "Negative"
    return "Neutral"

dataset_data = [
    {
        "instruction": "Detect the sentiment of the tweet.",
        "input": row_dict["tweet"],
        "output": sentiment_score_to_name(row_dict["sentiment"])
    }
    for row_dict in df.to_dict(orient="records")
]

In [15]:
dataset_data[0]

{'instruction': 'Detect the sentiment of the tweet.',
 'input': "@p0nd3ea Bitcoin wasn't built to live on exchanges.",
 'output': 'Positive'}

In [16]:
import json

In [17]:
with open("alpaca-bitcoin-sentiment-dataset.json", "w") as f:
    json.dump(dataset_data, f) 

In [18]:
from transformers import LlamaTokenizer, LlamaForCausalLM

In [19]:
BASE_MODEL =  "meta-llama/Llama-2-7b-hf"

model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="cpu",
    
)

tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
data =load_dataset("json", data_files="alpaca-bitcoin-sentiment-dataset.json")
#print(dataset_data)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [21]:
data["train"]

Dataset({
    features: ['input', 'instruction', 'output'],
    num_rows: 1897
})

In [22]:
CUTOFF_LEN = 256

In [23]:
def generate_prompt(data_point):    
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""

In [24]:
def tokenize(prompt, add_eos_token=True):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt

In [25]:
train_val = data["train"].train_test_split(
    test_size=200, shuffle=True, seed=42
)
train_data = (
    train_val["train"].shuffle().map(generate_and_tokenize_prompt)
)
val_data = (
    train_val["test"].shuffle().map(generate_and_tokenize_prompt)
)

Map:   0%|          | 0/1697 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [26]:
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]

BATCH_SIZE = 128
MICRO_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 300
OUTPUT_DIR = "experiments"

In [27]:
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
)

In [ ]:
model = prepare_model_for_int8_training(model)


/home/alourido/anaconda3/envs/alpaca/lib/python3.11/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [ ]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()